In [14]:
import pandas as pd
import import_ipynb
import pao_de_acucar as pda
import carrefour as crf
from tabulate import tabulate

In [2]:
_SUGGESTED = 1000
_MIN_PRICE = 1001
_MANUAL = 1002
_PDA = "Pão de Açúcar"
_CRF = "Carrefour"

markets = [_PDA, _CRF]

In [3]:
def selectResults(results, decision_mode=_SUGGESTED, manual_list=[]):
    if decision_mode == _SUGGESTED:
        selected = results.drop_duplicates(subset=['searchKey'], keep='first')
    elif decision_mode == _MIN_PRICE:
        selected = results.groupby('searchKey').min()
    elif decision_mode == _MANUAL:
        selected = results[results.index.isin(manual_list)]
    else:
        selected = results
    return selected

In [4]:
def fetchMarketItems(products, market):
    if market == _PDA:
        result = pda.fetchItems(products)
        result['market'] = _PDA
    elif market == _CRF:
        result = crf.fetchItems(products)
        result['market'] = _CRF
    return result

In [5]:
def getInput(accepted=[], case_sensitive=False, numeric_only=False):
    if numeric_only:
        txt = input()
        while not txt.isnumeric():
            print("Por favor insira um número válido!")
            txt = input()
    elif accepted == []:
        txt = input()
    else:
        txt = input()
        if case_sensitive:
            while txt not in accepted:
                print("Por favor insira uma resposta válida!")
                txt = input()
        else:
            accepted_l = [x.lower() for x in accepted] 
            while txt.lower() not in accepted_l:
                print("Por favor insira uma resposta válida!")
                txt = input()
    return txt

In [6]:
products = [
    "Atum solido Gomes da Costa natural",
    "Torrada Bauducco",
    "Suco Do Bem Goiaba",
    "Milho verde em conserva",
    "Pao de forma integral"
]

In [9]:
def queryMarkets(products):
    mkt_results = {}
    for m in markets:
        print ("Deseja buscar seus itens no mercado " + m + "? (s/n)")
        resp = getInput(accepted=['s', 'n', 'sim', 'não', 'nao'], case_sensitive=False).lower()
        if "s" in resp:
            mkt_results[m] = (fetchMarketItems(products, m))
    return mkt_results

In [15]:
def chooseItems(mkt_results, products):
    chosen = pd.DataFrame(columns=["market", "productName", "quantity", "price"])
    for p in products:
        temp = [mkt_results[m][mkt_results[m].searchKey==p] for m in mkt_results]
        pr = pd.concat(temp).reset_index().drop(columns=['index', 'searchKey'])
        print("Digite os índices dos itens que deseja incluir para o produto " + p.upper() + " (números separados por vírgula)")
        print(tabulate(pr, headers='keys', tablefmt='psql'))
        chosenItems = input().replace(" ", "").split(",")
        for i in chosenItems:
            chosen.loc[p+"_"+i] = pr.iloc[int(i)][['market', 'productName', 'quantity', 'price']]
    return chosen.reset_index().drop(columns=['index'])

In [19]:
def getAmounts(chosen):
    chosen['amount'] = 0
    for i, c in chosen.iterrows():
        print("Quantos de " + c.productName + " (R$ " + str(c.price).replace(".", ",") + ") você deseja comprar?")
        chosen.at[i,'amount'] = getInput(numeric_only=True)
        chosen['totalPrice'] = chosen.amount*chosen.price
    return chosen

In [18]:
def shop(products):
    mkt_results = queryMarkets(products)
    chosen_items = chooseItems(mkt_results, products)
    item_amounts = getAmounts(chosen_items)
    return item_amounts